In [ ]:
!pip install gradio requests

import gradio as gr
import requests
import random
from datetime import datetime

# Expanded genre-trope database
GENRE_TROPES = {
    "Fantasy": [
        "Chosen One", "Magic School", "Ancient Prophecy",
        "Quest for Artifact", "Dragons Return", "Fairy Curse"
    ],
    "Sci-Fi": [
        "Time Travel", "Alien Invasion", "Robot Uprising",
        "Dystopian Future", "First Contact", "Cyber Revolution"
    ],
    "Romance": [
        "Love Triangle", "Enemies to Lovers", "Fake Dating",
        "Second Chance", "Forbidden Love", "Matchmaking Mishap"
    ],
    "Horror": [
        "Haunted House", "Ancient Curse", "Isolated Cabin",
        "Possession", "Creepy Town Secret", "Living Doll"
    ],
    "Mystery": [
        "Amateur Detective", "Locked Room", "Cold Case",
        "Unreliable Narrator", "Wrongly Accused", "Hidden Treasure"
    ],
    "Adventure": [
        "Treasure Hunt", "Lost Civilization", "Survival Journey",
        "Heist Mission", "Race Against Time", "Shipwrecked"
    ]
}

def generate_story(genre, trope, word_count, creativity):
    API_URL = "https://api-inference.huggingface.co/models/mistralai/Mistral-7B-Instruct-v0.1"
    headers = {"Authorization": "Bearer YOUR_KEY_HERE"}  # Replace with your token
#DONT FORGET TO CHANGE THE KEY OR ELSE IT WONT WORK
    prompt = f"""Write a {word_count}-word {genre.lower()} story featuring the trope "{trope.lower()}".
    Requirements:
    - Exactly {word_count} words (+/- 10)
    - Professional {genre.lower()} writing style
    - Three-act structure
    - Vivid sensory descriptions
    - Character development
    - {creativity} level of creativity

    Story:"""

    response = requests.post(API_URL, headers=headers, json={"inputs": prompt})

    if response.status_code == 200:
        story = response.json()[0]['generated_text'].split("Story:")[-1].strip()
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"{genre}_{trope}_{timestamp}.txt"
        return f"✨ {genre} Story: {trope} ✨\n\n{story}", filename, story
    else:
        return f"Error: {response.text}", None, None

def update_tropes(genre):
    return gr.Dropdown(choices=GENRE_TROPES[genre], value=GENRE_TROPES[genre][0])

def random_selection():
    genre = random.choice(list(GENRE_TROPES.keys()))
    trope = random.choice(GENRE_TROPES[genre])
    return genre, trope

def save_story(story_text, filename):
    if filename:
        with open(filename, "w") as f:
            f.write(story_text)
        return f"Saved as {filename}"
    return "Could not save story"

# Gradio interface
with gr.Blocks(title="AI Story Generator", theme=gr.themes.Soft()) as app:
    gr.Markdown("""
    # 📖 Advanced AI Story Generator
    *Create custom stories with your preferred genre, tropes, and length!*
    """)

    with gr.Row():
        with gr.Column():
            genre_dropdown = gr.Dropdown(
                choices=list(GENRE_TROPES.keys()),
                label="Select Genre",
                value="Fantasy"
            )
            trope_dropdown = gr.Dropdown(
                choices=GENRE_TROPES["Fantasy"],
                label="Select Trope",
                value="Chosen One"
            )
            word_count = gr.Slider(100, 500, value=250, label="Word Count")
            creativity = gr.Radio(
                ["Standard", "Creative", "Experimental"],
                value="Standard",
                label="Creativity Level"
            )

            with gr.Row():
                generate_btn = gr.Button("Generate Story", variant="primary")
                random_btn = gr.Button("Surprise Me!", variant="secondary")

        with gr.Column():
            story_output = gr.Textbox(label="Your Story", lines=15)
            filename_display = gr.Textbox(label="Filename", visible=False)
            download_btn = gr.Button("Download Story", visible=False)
            save_status = gr.Markdown()

    # Interactions
    genre_dropdown.change(
        fn=update_tropes,
        inputs=genre_dropdown,
        outputs=trope_dropdown
    )

    generate_btn.click(
        fn=generate_story,
        inputs=[genre_dropdown, trope_dropdown, word_count, creativity],
        outputs=[story_output, filename_display, download_btn]
    )

    random_btn.click(
        fn=random_selection,
        inputs=[],
        outputs=[genre_dropdown, trope_dropdown]
    ).then(
        fn=generate_story,
        inputs=[genre_dropdown, trope_dropdown, word_count, creativity],
        outputs=[story_output, filename_display, download_btn]
    )

    download_btn.click(
        fn=save_story,
        inputs=[story_output, filename_display],
        outputs=save_status
    )

app.launch(share=True)  # Set share=False for local only

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://1758cc782f9e23e74e.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
